<a href="https://colab.research.google.com/github/JANANI-B-S/fake_news_detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn streamlit cloudflared


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import accuracy_score
# Load datasets
fake = pd.read_csv('/content/sample_data/fake.csv')
true = pd.read_csv('/content/sample_data/true.csv')

# Label datasets
fake['label'] = 0
true['label'] = 1

# Combine datasets
data = pd.concat([fake, true]).reset_index(drop=True)

# Shuffle data
data = data.sample(frac=1).reset_index(drop=True)

# Split features and labels
X = data['text']
y = data['label']

# Vectorize text
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_vectorized = vectorizer.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
# Save model and vectorizer
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)


Accuracy: 0.982739420935412


In [21]:
def predict_news(news):
    vectorized_news = vectorizer.transform([news])
    prediction = model.predict(vectorized_news)
    return "True News" if prediction[0] == 1 else "Fake News"

# User input for news
news_input = input("Enter the news text to check if it's Fake or True: ")

# Make prediction
result = predict_news(news_input)

# Display result
print("\nPrediction:", result)

Enter the news text to check if it's Fake or True: A woman with her hands behind her back surrounded by two officers, one shining a bright light at the camera, in a blurry image Woman sentenced in case that sparked Springfield cat-eating rumours

Prediction: Fake News


In [22]:
with open('app.py', 'w') as file:
    file.write("""
import streamlit as st
import pickle

# Load model and vectorizer
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

# Streamlit App
st.title("Fake News Detection")
st.write("Enter a news article below to check whether it's fake or real:")

news = st.text_area("News Text")

if st.button("Predict"):
    if not news.strip():
        st.error("Please enter some news text to predict.")
    else:
        vectorized_news = vectorizer.transform([news])
        prediction = model.predict(vectorized_news)
        result = "True News" if prediction[0] == 1 else "Fake News"
        st.success(f"Prediction: {result}")
    """)


In [23]:
!streamlit run app.py &>/content/logs.txt &
!npx cloudflared tunnel --url http://localhost:8501


⠙⠹⠸⠼⠴⠦⠧⠇2024-12-08T14:57:15Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2024-12-08T14:57:15Z INF Requesting new quick Tunnel on trycloudflare.com...
2024-12-08T14:57:19Z INF +--------------------------------------------------------------------------------------------+
2024-12-08T14:57:19Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2024-12-08T14:57:19Z INF |  https://moscow-adapter-tours-anticipated.tryc